<a href="https://colab.research.google.com/github/lkkcpaul/Gomoku/blob/main/playtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from scipy.special import softmax
import gomoku, player_basic

# for auto-reloading external modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Playtest

In [3]:
game = gomoku.Gomoku(15)
opponent = player_basic.FeaturePlayer('cp', -1) # human goes first, cp goes second

In [17]:
game.play(2,2) # human move
if not game.finished:
    game.play(*opponent.play(game)) # cp plays random move
else:
    print("game has ended")
game.show()

game has ended
   0 1 2 3 4 5 6 7 8 91011121314
 0 . . . . . . . . . . . . . . . 
 1 . . . . . ● . . . . . . . . . 
 2 ○ . ● . ○ ○ ○ . . . . . . . . 
 3 . ● ● ● ○ ● . . . . . . . . . 
 4 . . ● ○ ● ● . . . . . . . . . 
 5 . . ● ● ○ . . . . . . . . . . 
 6 . . ● ○ ● . . . . . . . . . . 
 7 . ○ ○ ○ ● ○ . . . . . . . . . 
 8 . . . . ○ . . . . . . . . . . 
 9 . . . . . . . . . . . . . . . 
10 . . . . . . . . . . . . . . . 
11 . . . . . . . . . . . . . . . 
12 . . . . . . . . . . . . . . . 
13 . . . . . . . . . . . . . . . 
14 . . . . . . . . . . . . . . . 


# Testing: PGFeaturePlayer

In [118]:
#Train against random player
p1 = player_basic.PGFeaturePlayer('player1', 1, epsilon=0.2, w_lr=0.01, theta_lr=0.01, discount=.8)
p2 = player_basic.RandomPlayer('rand_player',-1)
#p1.theta = np.array([1, 4, 9, 16, 1, 4, 9, 16, 0], dtype=float)
#p2.theta = p1.theta

In [119]:
for i in range(1000):
    if i%100 == 0:
        print(i, end=', ')
    game = gomoku.Gomoku(9)
    players = [p1, p2]
    turn = 0
    while not game.finished:
        game.play(*players[turn].play(game))
        turn = (turn+1)%2
    
    if game.winner == 1:
        p1_grads = p1.get_grad(game, game._moves_left)
    elif game.winner == -1:
        p1_grads = p1.get_grad(game, -game._moves_left)
    else:
        p1_grads = p1.get_grad(game, 0)
    if i%100 == 0:
        print(p1_grads)    
    p1.step(*p1_grads)
        

0, (array([-12.806836  ,  -2.45      ,   0.        ,   0.        ,
        -3.9148534 ,  -3.70316943,   0.        ,   0.        ,
       -14.88149144]), array([0., 0., 0., 0., 0., 0., 0., 0., 0.]))
100, (array([-2.88549986, -1.98759789, -0.12787601,  0.        , -3.88833589,
       -1.48582925, -0.48400375, -0.70592717, -2.67469625]), array([0.67133927, 0.44649022, 0.33891962, 0.29385298, 1.10572254,
       0.5275584 , 0.33080421, 0.38087839, 0.64418543]))
200, (array([4.01319983, 1.6421634 , 0.        , 0.6953784 , 5.01799396,
       2.26365876, 0.        , 0.        , 4.05578747]), array([-0.81514706, -0.16241913, -0.17135988, -0.13991703, -0.2235095 ,
       -0.22818868, -0.17452807, -0.17655494, -0.35651388]))
300, (array([-5.09644368, -3.01493161, -1.18124383,  0.        , -7.4601555 ,
       -0.2641727 , -0.22844445,  0.        , -4.785772  ]), array([0.09271699, 0.09790065, 0.13648414, 0.0901546 , 0.15587621,
       0.08994967, 0.11171161, 0.08956444, 0.1803092 ]))
400, (array([

In [120]:
print(p1.w)
print(p1.theta)

[-0.10389605  0.17102074  1.12208951  7.36330914  0.53080834 -0.33300721
 -0.19570932 -0.10682813  1.95806549]
[11.12417721  6.07329122  5.3946375   5.47957136 20.15556172  4.82738472
  4.71229543  4.54721331  8.93163274]


In [121]:
    game = gomoku.Gomoku(9)

In [125]:
game.play(*p1.play(game))
game.play(*p2.play(game))
game.show()

   0 1 2 3 4 5 6 7 8
 0 . . . ○ . . . . . 
 1 . . . . . . ○ ● . 
 2 . . . ○ . . ● . . 
 3 . . . . . . . . . 
 4 . ○ . . . . . . . 
 5 . . . . . . ● . . 
 6 . . . . . . . . . 
 7 . . . . . . . . . 
 8 . ● . . . . . . . 


In [95]:
p1 = player_basic.PGFeaturePlayer('player1', 0, epsilon=0.2, discount=.5)
p2 = player_basic.FeaturePlayer('cp',-1)
#p2.w = p1.w
p1.theta = np.array([1, 4, 9, 16, 1, 4, 9, 16, 0], dtype=float)
#p2.theta = p1.theta

In [23]:
for i in range(1000):
    if i%10 == 0:
        print(i, end=', ')
    game = gomoku.Gomoku(9)
    #p1color = np.random.choice([1, -1])
    #p1.piece = p1color
    p1.piece = 1
    #p2.piece = -p1color

    players = [p1, p2]
    turn = (1-p1color)//2
    while not game.finished:
        game.play(*players[turn].play(game))
        turn = (turn+1)%2
    
    if game.winner == p1color:
        p1_grads = p1.get_grad(game, game._moves_left)
        p2_grads = p2.get_grad(game, 0)
    elif game.winner == -p1color:
        p1_grads = p1.get_grad(game, 0)
        p2_grads = p2.get_grad(game, game._moves_left)
    else:
        p1_grads = p1.get_grad(game, 0)
        p2_grads = p2.get_grad(game, 0)
        
    p1.step(*p1_grads)
    p1.step(*p2_grads)
        

0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510, 520, 530, 540, 550, 560, 570, 580, 590, 600, 610, 620, 630, 640, 650, 660, 670, 680, 690, 700, 710, 720, 730, 740, 750, 760, 770, 780, 790, 800, 810, 820, 830, 840, 850, 860, 870, 880, 890, 900, 910, 920, 930, 940, 950, 960, 970, 980, 990, 

In [24]:
print(p1.w)
print(p1.theta)

[-0.44071582  0.57408265  4.95792967 59.04507414  0.79845849 -0.85027883
 -0.93484436  1.35433379  0.25558875]
[ 315.05927739  389.8059745   388.52052564  689.85896585 2120.74040209
  269.45710191  250.92240095  246.18682457  519.72712921]


In [25]:
game = gomoku.Gomoku(15)
p1.piece = +1
p2.piece = -1

In [26]:
game.play(7, 7)

In [36]:
game.play(*p1.play(game))
game.play(*p2.play(game))
game.show()

   0 1 2 3 4 5 6 7 8 91011121314
 0 . . . . . . . . . . . . . . . 
 1 . . . . . . . . . . . . . . . 
 2 . . . . . . . . . . . . . . . 
 3 . . . . . . . . . . . . . . . 
 4 . . . . . . . . . . . . . . . 
 5 . . . . . . . ● . . . . . . . 
 6 . . . . . . ● ● . . . . . . . 
 7 . . . . ○ ○ ● ● ● . . . . . . 
 8 . . . ○ ○ ○ ● ● ● . . . . . . 
 9 . . . ○ ○ ○ ● ● . . . . . . . 
10 . . . . ○ ○ . . . . . . . . . 
11 . . . . . . . . . . . . . . . 
12 . . . . . . . . . . . . . . . 
13 . . . . . . . . . . . . . . . 
14 . . . . . . . . . . . . . . . 


In [37]:
p1 = player_basic.PGFeaturePlayer('player1', 0)
p2 = player_basic.FeaturePlayer('player2', 0)

In [ ]:
for i in range(1000):
    if i%10 == 0:
        print(i, end=', ')
    game = gomoku.Gomoku(9)
    p1color = np.random.choice([1, -1])
    p1.piece = p1color
    p2.piece = -p1color

    players = [p1, p2]
    turn = (1-p1color)//2
    while not game.finished:
        game.play(*players[turn].play(game))
        turn = (turn+1)%2
    
    if game.winner == p1color:
        p1_grads = p1.get_grad(game, 1.)
        #p2_grads = p2.get_grad(game, -1.)
    elif game.winner == -p1color:
        p1_grads = p1.get_grad(game, -1.)
        #p2_grads = p2.get_grad(game, 1.)
    else:
        p1_grads = p1.get_grad(game, 0.)
        #p2_grads = p2.get_grad(game, 0.)
        
    p1.step(*p1_grads)
    #p1.step(*p2_grads)
        

0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510, 520, 530, 540, 550, 560, 570, 580, 590, 600, 610, 620, 630, 640, 650, 660, 670, 680, 690, 700, 710, 720, 730, 740, 750, 760, 770, 780, 790, 800, 810, 820, 830, 840, 850, 860, 870, 880, 890, 900, 910, 920, 930, 940, 950, 960, 970, 980, 990, 

In [ ]:
print(p1.w)
print(p1.theta)

[-0.15362757 -0.03364655  0.          0.         -0.0740043  -0.00073599
 -0.03723498 -0.00846379 -0.62696604]
[-0.83542265 -3.15222877 -3.1762802  -3.17800188 -0.48550833 -2.91121014
 -3.05746383 -3.03022919 -6.35600376]


In [ ]:
game = gomoku.Gomoku(9)
p1.piece = +1
p2.piece = -1

In [ ]:
game.play(*p1.play(game))
game.play(*p2.play(game))
game.show()

[0.01234568 0.01234568 0.01234568 0.01234568 0.01234568 0.01234568
 0.01234568 0.01234568 0.01234568 0.01234568 0.01234568 0.01234568
 0.01234568 0.01234568 0.01234568 0.01234568 0.01234568 0.01234568
 0.01234568 0.01234568 0.01234568 0.01234568 0.01234568 0.01234568
 0.01234568 0.01234568 0.01234568 0.01234568 0.01234568 0.01234568
 0.01234568 0.01234568 0.01234568 0.01234568 0.01234568 0.01234568
 0.01234568 0.01234568 0.01234568 0.01234568 0.01234568 0.01234568
 0.01234568 0.01234568 0.01234568 0.01234568 0.01234568 0.01234568
 0.01234568 0.01234568 0.01234568 0.01234568 0.01234568 0.01234568
 0.01234568 0.01234568 0.01234568 0.01234568 0.01234568 0.01234568
 0.01234568 0.01234568 0.01234568 0.01234568 0.01234568 0.01234568
 0.01234568 0.01234568 0.01234568 0.01234568 0.01234568 0.01234568
 0.01234568 0.01234568 0.01234568 0.01234568 0.01234568 0.01234568
 0.01234568 0.01234568 0.01234568]
   0 1 2 3 4 5 6 7 8
 0 . . . . . . . . . 
 1 . . . . . . . . . 
 2 . . . . . . . . . 
 3 . . 

In [ ]:
p1.theta

array([-0.83542265, -3.15222877, -3.1762802 , -3.17800188, -0.48550833,
       -2.91121014, -3.05746383, -3.03022919, -6.35600376])

In [ ]:
p1.get_features(game) @ p1.theta

array([[-6.35600376, -6.35600376, -6.35600376, -6.35600376, -6.35600376,
        -6.35600376, -6.35600376, -6.35600376, -6.35600376],
       [-6.35600376, -6.35600376, -6.35600376, -6.35600376, -6.35600376,
        -6.35600376, -6.35600376, -6.35600376, -6.35600376],
       [-6.35600376, -6.35600376, -6.35600376, -6.35600376, -6.35600376,
        -6.35600376, -6.35600376, -6.35600376, -6.35600376],
       [-6.35600376, -6.35600376, -6.35600376, -6.35600376, -6.35600376,
        -6.35600376, -7.1914264 , -7.1914264 , -7.1914264 ],
       [-6.35600376, -6.35600376, -6.35600376, -6.35600376, -6.35600376,
        -6.35600376, -7.67693473, 26.18234874, -7.67693473],
       [-6.35600376, -6.35600376, -6.35600376, -6.35600376, -6.35600376,
        -6.35600376, -7.67693473, 26.18234874, -7.67693473],
       [-6.35600376, -6.35600376, -6.35600376, -6.35600376, -6.35600376,
        -6.35600376, -6.84151208, -6.84151208, -6.84151208],
       [-6.35600376, -6.35600376, -6.35600376, -6.35600376, -6

In [ ]:
p1.theta

array([-0.83542265, -3.15222877, -3.1762802 , -3.17800188, -0.48550833,
       -2.91121014, -3.05746383, -3.03022919, -6.35600376])